# Replication of Fama-French

Follow along tidy finance : [Link](https://www.tidy-finance.org/python/replicating-fama-and-french-factors.html)

In [1]:
# Packages
import pandas as pd
import numpy as np
import sqlite3
import statsmodels.formula.api as smf

# For FF3 data
import pandas_datareader as pdr

# For outputting regression tables
from regtabletotext import prettify_result

In [2]:
start_date = "1960-01-01"
end_date   = "2023-12-31"

In [3]:
# Download from the web
factors_ff3_monthly_raw = pdr.DataReader(
    name        = "F-F_Research_Data_Factors",
    data_source = "famafrench",
    start       = start_date,
    end         = end_date
)[0]

# Scale, reset index, etc.
factors_ff3_monthly = (factors_ff3_monthly_raw
  .divide(100)
  .reset_index(names="month")
  .assign(month=lambda x: pd.to_datetime(x["month"].astype(str)))
  .rename(str.lower, axis="columns")
  .rename(columns={"mkt-rf": "mkt_excess"})
)

/var/folders/fw/ccvwsj857433zz3m609ks6h00000gn/T/ipykernel_4450/3144967270.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  factors_ff3_monthly_raw = pdr.DataReader(
/var/folders/fw/ccvwsj857433zz3m609ks6h00000gn/T/ipykernel_4450/3144967270.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  factors_ff3_monthly_raw = pdr.DataReader(


In [4]:
factors_ff3_monthly.head()

,month,mkt_excess,smb,hml,rf
0,1960-01-01,-0.0698,0.0209,0.0278,0.0033
1,1960-02-01,0.0117,0.0051,-0.0193,0.0029
2,1960-03-01,-0.0163,-0.0049,-0.0294,0.0035
3,1960-04-01,-0.0171,0.0032,-0.0228,0.0019
4,1960-05-01,0.0312,0.0121,-0.0370,0.0027


In [8]:
# Set up database
# connection = sqlite3.connect("data/test.db")
tidy_finance = sqlite3.connect(database="../data/tidy_finance_python.sqlite")

(factors_ff3_monthly
 .to_sql(name      = "factors_ff3_monthly",
         con       = tidy_finance,
         if_exists = "replace",
         index     = False)
 )


768

In [9]:
print(tidy_finance.total_changes)

768


In [10]:
# In the future, if you want to read in table to memory
pd.read_sql_query(
    sql         = "SELECT month, rf FROM factors_ff3_monthly",
    con         = tidy_finance,
    parse_dates = {"month"}
)

,month,rf
0,1960-01-01,0.0033
1,1960-02-01,0.0029
2,1960-03-01,0.0035
3,1960-04-01,0.0019
4,1960-05-01,0.0027
...,...,...
763,2023-08-01,0.0045
764,2023-09-01,0.0043
765,2023-10-01,0.0047
766,2023-11-01,0.0044


In [12]:
import yfinance as yf

prices = (yf.download(
    tickers="AAPL", 
    start="2000-01-01", 
    end="2022-12-31", 
    progress=False
  )
  .reset_index()
  .assign(symbol="AAPL")
  .rename(columns={
    "Date": "date", 
    "Open": "open", 
    "High": "high",
    "Low": "low",
    "Close": "close", 
    "Adj Close": "adjusted", 
    "Volume": "volume"}
  )
)

In [18]:
prices = yf.download(
    tickers="AAPL", 
    start="2000-01-01", 
    end="2022-12-31", 
    progress=False
  ).reset_index().assign(symbol="AAPL")



prices.head().round(3)

,Date,Open,High,Low,Close,Adj Close,Volume,symbol
0,2000-01-03,0.936,1.004,0.908,0.999,0.846,535796800,AAPL
1,2000-01-04,0.967,0.988,0.903,0.915,0.775,512377600,AAPL
2,2000-01-05,0.926,0.987,0.920,0.929,0.786,778321600,AAPL
3,2000-01-06,0.948,0.955,0.848,0.848,0.718,767972800,AAPL
4,2000-01-07,0.862,0.902,0.853,0.888,0.752,460734400,AAPL


,date,open,high,low,close,adjusted,volume,symbol
0,2000-01-03,0.936,1.004,0.908,0.999,0.846,535796800,AAPL
1,2000-01-04,0.967,0.988,0.903,0.915,0.775,512377600,AAPL
2,2000-01-05,0.926,0.987,0.920,0.929,0.786,778321600,AAPL
3,2000-01-06,0.948,0.955,0.848,0.848,0.718,767972800,AAPL
4,2000-01-07,0.862,0.902,0.853,0.888,0.752,460734400,AAPL
